In [41]:
from manim import *
import numpy as np
from sympy import symbols, cos, sin, pi, lambdify, diff, Matrix
from sympy.vector import CoordSys3D, gradient, curl
import sympy as S

<center> <strong>Basic Wave Equation </strong></center>

$$
\begin{align*}
P(x) &= A\sin{\left(\frac{2\pi}{\lambda} \left( x + \nu t\right) + \phi\right)} \\
\text{where...}\\
P(x) &= \text{Pressure (defined only by an x coordinate)}\\
A &= \text{Amplitude}\\
\lambda &= \text{Wavelength}\\
x &= \text{Position along the x axis}\\
\text{and...}\\
\nu &= f\lambda\\
f &= \text{frequency} = \frac{1}{T}\\
T &= \text{period}\\
t &= \text{time}\\
\end{align*}
$$

In [42]:
t, v, A, lamda, phi, omega, Omega = S.symbols('t, v, A, lamda, phi, omega, Omega' )

a = 5
lamma = 10
phi_value = 0
seconds_in_day = T=  24*60*60
omega=2*PI/seconds_in_day

substitutions_list = [(A, a),(Omega, omega),(lamda, lamma), (phi, phi_value)]

In [43]:
#lets define some pressure field only based on the x axis


#first we define a coordinate system
N = CoordSys3D('N')
N.x

N.x

In [44]:
#interesting... lets try
N.i

N.i

Cool, so 'N.x' gives us the magnitude of the basis vector in the x direction. 'N.i' gives us the base vector 

Let's define a spherical coordinate system 'B' to be compatible with N

In [45]:
v_e = 3
v_i = 10
v_n = 4
v_u = 0

In [46]:
Omega = omega*cos(phi)*N.j + omega*sin(phi)*N.k
v = v_e*N.i + v_n*N.j + v_u*N.k
v

3*N.i + 4*N.j

In [47]:
Omega

(7.27220521664304e-5*cos(phi))*N.j + (7.27220521664304e-5*sin(phi))*N.k

In [48]:
-Omega.cross(v)

(0.000290888208665722*sin(phi))*N.i + (-0.000218166156499291*sin(phi))*N.j + (0.000218166156499291*cos(phi))*N.k

In [49]:
B = N.create_new('B', transformation='spherical')
B

B

In [50]:
B.i

B.i

In [51]:
N.i.dot(B.i)

1

In [52]:
B.k

B.k

In [53]:
B.z

AttributeError: 'CoordSys3D' object has no attribute 'z'

In [54]:
B.theta, B.phi

(B.theta, B.phi)

In [55]:
rho = Matrix([[0],[2*S.pi/seconds_in_day],[0]])
type(rho)

sympy.matrices.dense.MutableDenseMatrix

lets define a velocity field with this x,y,z coordinate system

In [56]:
turd_velocity_field = N.x*N.i + N.y*N.j
turd_cori_acc = -Omega.cross(turd_velocity_field)
turd_eval = turd_cori_acc.subs(substitutions_list)
turd_matrix = turd_eval.to_matrix(N)
turd_matrix

Matrix([
[                      0],
[                      0],
[7.27220521664304e-5*N.x]])

In [57]:
turd_func= lambdify((N.x, N.y), turd_matrix, 'numpy')

In [58]:
def give_manim_goodies(sympy_vector_field, subs_list, coords): #coords should be N or whatever the base coordinate system is
    manim_string = S.latex(sympy_vector_field).replace('}}', '} }'). replace('{{', '{ {')
    evaluated_vec_field = sympy_vector_field.subs(substitutions_list)
    vec_field_as_matrix = evaluated_vec_field.to_matrix(coords)
    vec_field_as_func = lambdify((N.x, N.y), vec_field_as_matrix, 'numpy')

    return (manim_string, vec_field_as_func)

def coriolis_force(velocity_field):
    Omega = omega*cos(phi)*N.j + omega*sin(phi)*N.k
    return -2*Omega.cross(velocity_field)
    
    
    

In [59]:
velocity_field = (A*sin(N.x*2*S.pi/lamda)+v_i)*N.i + N.x/lamda*N.j 
velocity_field

(A*sin(2*N.x*pi/lamda) + 10)*N.i + N.x/lamda*N.j

In [60]:
coriolis_force(velocity_field)

(0.000145444104332861*N.x*sin(phi)/lamda)*N.i + (-0.000145444104332861*(A*sin(2*N.x*pi/lamda) + 10)*sin(phi))*N.j + (-2*(-7.27220521664304e-5*A*sin(2*N.x*pi/lamda) - 0.000727220521664304)*cos(phi))*N.k

In [61]:
give_manim_goodies(coriolis_force(turd_velocity_field), substitutions_list, N)[1]

<function _lambdifygenerated(Dummy_15425, Dummy_15424)>

In [62]:
coriolis_acceleration = -Omega.cross(velocity_field)
2*coriolis_acceleration

(0.000145444104332861*N.x*sin(phi)/lamda)*N.i + (-0.000145444104332861*(A*sin(2*N.x*pi/lamda) + 10)*sin(phi))*N.j + (-2*(-7.27220521664304e-5*A*sin(2*N.x*pi/lamda) - 0.000727220521664304)*cos(phi))*N.k

In [63]:
substitutions_list

[(A, 5), (Omega, 7.27220521664304e-05), (lamda, 10), (phi, 0)]

In [64]:
phi_value = S.pi/4
substitutions_list[3]=(phi, phi_value)
substitutions_list

coriolis_eval = coriolis_acceleration.subs(substitutions_list)
coriolis_eval

(3.63610260832152e-6*sqrt(2)*N.x)*N.i + (sqrt(2)*(-0.000363610260832152*sin(N.x*pi/5) - 0.000727220521664304)/2)*N.j + (sqrt(2)*(0.000363610260832152*sin(N.x*pi/5) + 0.000727220521664304)/2)*N.k

In [65]:
coriolis_acceleration_latex_spaced = S.latex(coriolis_acceleration).replace('}}', '} }'). replace('{{', '{ {')

In [66]:
cori_in_the_house = coriolis_eval.to_matrix(N)
cori_in_the_house

Matrix([
[                                       3.63610260832152e-6*sqrt(2)*N.x],
[sqrt(2)*(-0.000363610260832152*sin(N.x*pi/5) - 0.000727220521664304)/2],
[ sqrt(2)*(0.000363610260832152*sin(N.x*pi/5) + 0.000727220521664304)/2]])

In [67]:
cori_in_the_house[0]

3.63610260832152e-6*sqrt(2)*N.x

In [68]:
coriolis_vector_func = lambdify((N.x, N.y), cori_in_the_house, 'numpy')
coriolis_vector_func

<function _lambdifygenerated(Dummy_15427, Dummy_15426)>

In [27]:
pressure_gradient_function([array,array])

NameError: name 'pressure_gradient_function' is not defined

In [28]:
field_equation_latex

NameError: name 'field_equation_latex' is not defined

In [29]:
coriolis_vector_func(0,0).flatten()

array([ 0.        , -0.00051422,  0.00051422])

In [30]:
substitutions_list

[(A, 5), (Omega, 7.27220521664304e-05), (lamda, 10), (phi, pi/4)]

In [40]:
%%manim -qm -v  WARNING Coriolis_field
class Coriolis_field(MovingCameraScene):
    def construct(self):

        # def dot_position(mobject):
        #     mobject.set_value(dot.get_center()[0])
        #     mobject.next_to(dot)
        domain = np.array([[-15, 15, 0.5],[-15, 15, 0.5],[-10, 10, 1]])
        
        # print(np.min(domain[0]))
        field_1_function = give_manim_goodies(turd_velocity_field, substitutions_list, N)
        field_2_function = give_manim_goodies(coriolis_force(turd_velocity_field), substitutions_list, N)

        print(type(field_1_function))
        # x_var = Variable(np.min(domain[0]), 'x', num_decimal_places=3)
        equations_mobject = MathTex(field_1_function[0] +'\\'+ field_2_function[0]).to_corner(UL)
        self.add(equations_mobject)
        # field_function = Variable(np.min(domain[0]), equation, num_decimal_places=3).next_to(x_var, RIGHT)

        

        # self.add_fixed_in_frame_mobjects(text)
        
       
        first_arrow_field_function = lambda pos: func= field_1_function[1](pos[0], pos[1]).flatten()
        # domain = np.array([[-15, 15, 1],[-10, 10, 1],[-40, 40, 0.5]])
        coriolis_vector_field = ArrowVectorField(first_arrow_field_function, 
                                  x_range= [-15,15,1],
                                  y_range= [-15,15,1])
        second_arrow_field_function = lambda pos: func= field_2_function[1](pos[0], pos[1]).flatten()
        velocity_vector_field = ArrowVectorField(second_arrow_field_function, 
                                  x_range= [-15,15,1],
                                  y_range= [-15,15,1])
        # sets dot at bottom left of screen
        # dot = Dot().move_to([np.minimum(domain[0]), np.minimum(domain[1]),0]to
        
        
        # field_function.add_updater(lambda v: v.tracker.set_value(func([x_var.tracker.get_value(),0])[0]))
        #keeps the equation handy
        
        #the thing we want
        self.add(coriolis_vector_field, velocity_vector_field)
        
        
        # self.play(self.camera.auto_zoom(vector_field, margin=1), run_time=0.5)
        # text = VGroup(x_var, field_function)
        # text.scale(0.75).to_corner(UL)
        
        # self.play(self.camera.set_zoom(self.camera.get_zoom()*0.25), runtime=5)
        # self.camera.set_zoom(.5)
        # frame = self.camera
        
    
        
        # dot = Dot().move_to(RIGHT*(np.min(domain[0])))
        # self.play(Create(dot))
        # dot.add_updater(lambda d: d.next_to(x_var.tracker.get_value()*RIGHT, UP))


        
        # self.play(x_var.tracker.animate.set_value(5), run_time=10, rate_func=linear)

        # dot = Dot()
        # self.add()
        
        # self.play(MoveToTarget(equation)
        # self.wait(2)
        
        

SyntaxError: cannot assign to lambda (<string>, line 24)

In [34]:
# %load_ext line_profiler
%prun Coriolis_field().construct()

         41143868 function calls (35939973 primitive calls) in 75.870 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
3651800/682262    3.356    0.000    3.953    0.000 traversal.py:119(_preorder_traversal)
   109554    3.309    0.000    7.102    0.000 facts.py:599(deduce_all_facts)
5950760/5933462    2.320    0.000    2.402    0.000 {built-in method builtins.isinstance}
686154/641948    1.881    0.000   12.144    0.000 cache.py:69(wrapper)
71114/59582    1.874    0.000    7.428    0.000 basisdependent.py:238(__new__)
   361336    1.662    0.000    3.410    0.000 sympify.py:101(sympify)
    59551    1.578    0.000    1.578    0.000 bezier.py:54(<lambda>)
  2139186    1.450    0.000    1.818    0.000 facts.py:633(<genexpr>)
124930/21142    1.227    0.000   21.762    0.001 printer.py:294(_print)
  2330430    1.158    0.000    1.366    0.000 {built-in method builtins.hasattr}
   126852    1.083    0.000    2.265    0.000 repma